In [16]:
from arcgis.gis import GIS
try:
    from ujson import loads
except ModuleNotFoundError:
    from json import loads


# load credentials
f = open('credentials.json', "r")
credentials = loads(f.read())
# Closing file
f.close()

portal = credentials['portal']
username = credentials['username']
password = credentials['password']

gis = GIS(portal, username, password, verify_cert=False)

List Users in Org

In [17]:
#  users = [u.username for u in gis.users.search(query='*')]
users = gis.users.search(max_users=1000)
print("detected {0} users in {1} org".format(len(users), portal))
print(users)


detected 181 users in https://ivt.maps.arcgis.com org
[<User username:a.giles_IVT>, <User username:aazerki_IVT>, <User username:ABong_IVT>, <User username:achapkowski_IVT>, <User username:acrawford_IVT>, <User username:adoan@esri.com>, <User username:adoshi_IVT>, <User username:aevenpaz_IVT>, <User username:ALeason_IVT>, <User username:alohec@esri.com>, <User username:amason2_IVT>, <User username:amason_IVT>, <User username:ameriaux_IVT>, <User username:ankit_IVT>, <User username:arobledo_IVT>, <User username:atecpoyotl_IVT>, <User username:bconnolly_IVT>, <User username:BenkelmanIVT>, <User username:bfilippin_IVT>, <User username:bgerlt_IVT>, <User username:blegeer_IVT>, <User username:boneill_ivt>, <User username:BPiersol_IVT>, <User username:camilo.cortes_IVT>, <User username:CAndrews_IVT>, <User username:carrera_ivt>, <User username:cbeck_IVT>, <User username:ccardona_imagery>, <User username:cchavis_IVT>, <User username:ccortes_IVT>, <User username:cerikson_IVT>, <User username:cf

List All Users Privileges

In [18]:
privileges = [[u.username, u.privileges] for u in users]
print(privileges)


'\nprivileges = [[u.username, u.privileges] for u in users]\nprint(privileges)\n'

List all Users Items

In [ ]:
from tqdm.notebook import tqdm # For Status Bar


def get_most_recent_usage_date(item_info):
    try:
        my_info = item_info.usage(date_range='1Y', as_df=False)
        for n in range(len(my_info)):
            i = my_info[n]
            if i['data']:
                return i['startTime']
            if not i['data'] and n == len(my_info):
                return None
    except IndexError:
        return None


def get_items(user, get_usage=False):
    list_items = []
    user_items = user.items()
    for item in user_items:
        item_id = item.itemid
        item_info = gis.content.get(item_id)
        accessibility = item_info.access
        if get_usage:
            recent_usage = get_most_recent_usage_date(item_info)
            list_items.append([item.title, item.type, item_id, accessibility, recent_usage])
        else:
            list_items.append([user.username, item.title, item.type, item_id, accessibility])
    return list_items


def get_all_items(users, get_usage=False):
    master_list = []
    for i in tqdm(range(len(users))):
        user = users[i]
        item_list = get_items(user, get_usage)
        master_list.append(item_list)
    return master_list


''' Note enabling get_usage takes a long... long... real long time to process as every item must be queried individually '''
all_items = get_all_items(users, get_usage=False)

Convert items to pandas dataframe and export as csv

In [ ]:
out_csv = r'fill_in_your_path_and_file_name_here.csv' # Just set output file

df = pd.DataFrame(all_items)
df.to_csv(out_csv, index=False)

print("output csv to {}".format(out_csv))